# Protein design pipeline

This is a pipeline for de novo protein design, incorporating the EvoDiff diffusion model for sequence generation, and OmegaFold model for folding prediciton. This was adapted from their Github repos (see below) for the S2DS Summer 2024 course, Team Polyploy. It's been optimized to work in Google Colab.

EvoDiff:

Diffusion model for de novo generation of protein sequences (outputs are fastA files). It can generate proteins with disordered regions, which is an advatage in relation to structure-based models (like RFDiffusion), while maintaining the capabiling of scaffold motif design.

https://github.com/microsoft/evodiff


OmegaFold:

The first computational method to successfully predict high-resolution protein structure from a single primary sequence alone. Using a new combination of a protein language model that allows us to make predictions from single sequences and a geometry-inspired transformer model trained on protein structures, OmegaFold outperforms RoseTTAFold and achieves similar prediction accuracy to AlphaFold2 on recently released structures.

https://github.com/HeliXonProtein/OmegaFold



## Connecting to S2DS Gihub Repo (~2min)

Cloning github repo is not stricly necessary, unless you want to access specific files other than this script. But the script itself can run on it's own without access to other files in the github repo. Can skip to "Get reference protein" if not cloning the repo.

To clone the git repo, use private ssh keys to access it. The key generation step only needs to be done once (1st and 2nd cell; now commented out).

In [ ]:
# #first command generates key rsa 4096 long. Puts your email at the end of it, save the public key to id_rsa and does all of this with no passphrase being asked
# !ssh-keygen -t rsa -b 4096 -C "youremail" -f /root/.ssh/id_rsa -N ""

# #this command scans GitHub's SSH key and adds it to the known_hosts file, which is used to verify the identity of the remote server.
# !ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# #print the private key that you need to copy and paste in github (the key is the last line in the output which starts with "ssh-rsa" and ends with your email included)
# !cat /root/.ssh/id_rsa.pub


Add the public SSH key to GitHub (one-time setup):
1. go to your GitHub account settings and navigate to the “SSH and GPG keys” section
2. Click “New SSH key” and provide a name for your key, e.g. my_colab_key
3. Copy the key that appears as the last line in the output of the previous cell (starts with "ssh-rsa" and ends with your email) and paste it into the key text box on GitHub
4. Click “Add SSH key” and refresh github


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !mkdir /content/drive/MyDrive/ssh_colab
# !cp /root/.ssh/* /content/drive/MyDrive/ssh_colab
# !ls /content/drive/MyDrive/ssh_colab


Mounted at /content/drive
id_rsa	id_rsa.pub  known_hosts


Now from google drive we export the key to connect google colab with github, which will allow use of desired git commands (git-clone, git-commit, etc) and access to all files in the github repo through the google colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir /root/.ssh/
!cp /content/drive/MyDrive/ssh_colab/* /root/.ssh/

Mounted at /content/drive


Note: An alternative to this Google Drive SSH authentication method, which has been tested and shown to work, can be found here, if desired:
https://felixbmuller.medium.com/connect-a-private-github-repository-with-google-colab-via-a-deploy-key-cca8ad13007


Git clone the repository and switch to your branch

In [2]:
import os

#Only clone the repository if it doesn't already exist (to avoid duplicates/complications)
if ~os.path.isdir("S2DS-Summer24-Polyploy"):
    print('S2DS Polyploy directory does not exist, cloning from github')

    !git clone git@github.com:S2DSLondon/S2DS-Summer24-Polyploy.git


S2DS Polyploy directory does not exist, cloning from github
Cloning into 'S2DS-Summer24-Polyploy'...
remote: Enumerating objects: 1896, done.
remote: Counting objects: 100% (681/681), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 1896 (delta 477), reused 531 (delta 392), pack-reused 1215
Receiving objects: 100% (1896/1896), 118.11 MiB | 21.05 MiB/s, done.
Resolving deltas: 100% (1021/1021), done.
Updating files: 100% (332/332), done.


Switch to the branch you would like to work in

In [3]:
%cd S2DS-Summer24-Polyploy
#change to branch of interest
!git checkout public

/content/S2DS-Summer24-Polyploy
Branch 'public' set up to track remote branch 'public' from 'origin'.
Switched to a new branch 'public'


# Get reference protein


We need to provide a pdb file of the protein from where our motif is kept. Below we will automatically dowload it and save it to the desired directory.

In [24]:
import os
import requests

def download_pdb(pdb_code, pdb_file_path):
    url = f"https://files.rcsb.org/download/{pdb_code.upper()}.pdb"
    response = requests.get(url)

    if response.status_code == 200:
        os.makedirs(os.path.dirname(pdb_file_path), exist_ok=True)  # Create directories if not present
        with open(pdb_file_path, 'wb') as f:
            f.write(response.content)
        print(f"PDB file {pdb_code} downloaded successfully to {pdb_file_path}")
    else:
        print(f"Failed to download PDB file {pdb_code}. Status code: {response.status_code}")

# Variables to define for evodiff
pdb_code = '1SMD'  # @param {type:"string"}
num_seqs = 1  # @param {type:"integer"}
data_top_dir = 'data/external/pdb/'

# Variables to define for extracting motifs
pdb_file_name = f'{pdb_code}.pdb'
pdb_file_path = os.path.join(data_top_dir, 'scaffolding-pdbs', pdb_file_name)

# Download the PDB file from online
download_pdb(pdb_code, pdb_file_path)

# Check if the file exists
if os.path.exists(pdb_file_path):
    print(f"PDB file found: {pdb_file_path}")
else:
    print(f"PDB file not found: {pdb_file_path}")


PDB file 1SMD downloaded successfully to data/external/pdb/scaffolding-pdbs/1SMD.pdb
PDB file found: data/external/pdb/scaffolding-pdbs/1SMD.pdb


 Important: Here you need to inspect the PBD file to see if the first residues starts at 1 or not. If it doesnt then we need to add an offset so the correct residues are extracted. At the moment this needs to be done manually, but in future this could be automated.

 Provide the start residues for the motif you are interested in scaffolding (the end residue is inclusive) and the offset if there is one. The adjusted start/end residue numbers for evodiff should be the motif site start/end minus the offset.



In [25]:
# Function to create adjusted indices based on the number of motifs
def adjust_indices(motif_start, motif_end, offset):
    start_idx = motif_start - offset
    end_idx = motif_end - offset
    return start_idx, end_idx

# Parameters
motif_start = 212  # @param {type:"integer"}
motif_end = 233    # @param {type:"integer"}
offset = 16        # @param {type:"integer"}

# Get adjusted indices
start_idx, end_idx = adjust_indices(motif_start, motif_end, offset)

print(f"Adjusted start_residue_number for evodiff: {start_idx}")
print(f"Adjusted end_residue_number for evodiff: {end_idx}")



Adjusted start_residue_number for evodiff: 196
Adjusted end_residue_number for evodiff: 217


Now we will extract the motif to be kept as a pdb file, for later comparison with the generated protein.

In [26]:
# Install Biopython if not already installed
!pip install biopython

from Bio import PDB
from Bio.SeqUtils import seq1
import os
import warnings
warnings.simplefilter('ignore', PDB.PDBExceptions.PDBConstructionWarning)

# Parse the PDB file
parser = PDB.PDBParser()

# Parse the PDB file
structure = parser.get_structure('protein', pdb_file_path)

# Create a new structure object to store the extracted residues
new_structure = PDB.Structure.Structure('new_structure')

# Store the sequence of the extracted motif
motif_sequence = []

for model in structure:
    new_model = PDB.Model.Model(model.id)  # Create a new model
    for chain in model:
        new_chain = PDB.Chain.Chain('A')  # Create a new chain (adjust 'A' as needed)
        for residue in chain.get_residues():
            residue_id = residue.get_id()[1]  # Get the residue number
            if motif_start <= residue_id <= motif_end:
                # Create a new residue object and add it to the new chain
                new_residue = PDB.Residue.Residue(residue.get_id(), residue.resname, residue.segid)
                for atom in residue:
                    new_residue.add(atom)
                new_chain.add(new_residue)

                # Append the residue's one-letter code to the motif sequence
                motif_sequence.append(seq1(residue.resname))

        # Add the new chain to the new model
        new_model.add(new_chain)

    # Add the new model to the new structure
    new_structure.add(new_model)
# Define the path to the output PDB file
output_pdb_file = 'data/interim/test_extracted_motif.pdb' #@param {type:"string"}

# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(output_pdb_file), exist_ok=True)

# Save the new structure to a PDB file
io = PDB.PDBIO()
io.set_structure(new_structure)
io.save(output_pdb_file)

# Convert the motif sequence list to a string
motif_sequence_str = ''.join(motif_sequence)

print(f"Extracted PDB file saved to: {output_pdb_file}")
print(f"Motif sequence: {motif_sequence_str}")


Extracted PDB file saved to: data/interim/test_extracted_motif.pdb
Motif sequence: DKLHNLNSNWFPEGSKPFIYQE


# EvoDiff


First step is to generate a protein sequence (fastA file), using the Evodiff diffusion model.

Install required libraries as below:

In [23]:
!pip install evodiff
!pip install git+https://github.com/microsoft/evodiff.git

!pip install torch
!pip install torch-geometric
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.3.0+cpu.html



  Cloning https://github.com/microsoft/evodiff.git to /tmp/pip-req-build-4qtan4ou
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/evodiff.git /tmp/pip-req-build-4qtan4ou
  Resolved https://github.com/microsoft/evodiff.git to commit f813abd06baa3e31098b46c67dd2d3f0cbc65b7d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in links: https://data.pyg.org/whl/torch-2.3.0+cpu.html




 We need to download model information from zenodo. For demonstration purposes, we show an example using the lightest model - 38M model here, and generation on a CPU.
[EvoDiff has a bunch of different models, some require more memory and GPU.]


Next, we run the generation and can specify a few things:

* `scaffold_length` we want to generate. The code will randomly sample the location of the motif within the specified scaffold length.

* number of sequences to be designed under: `batch_size`=1

* whether to run on cpu/gpu etc under: `device`='cpu'


In [28]:
#load the model
from evodiff.pretrained import OA_DM_38M

checkpoint = OA_DM_38M()
model, collater, tokenizer, scheme = checkpoint


from evodiff.conditional_generation import generate_scaffold

scaffold_length = 150 #@param {type:"integer"}
batch_size = 2 #@param {type:"integer"}
device = "cpu" #@param{type:"string"}

# Make sure start_idx and end_idx are lists, even if they contain a single element
start_idx = [start_idx]  # Assuming start_idx is an integer representing the start index
end_idx = [end_idx]  # Assuming end_idx is an integer representing the end index

generated_sequence, new_start_idx, new_end_idx = generate_scaffold(model, pdb_code, start_idx, end_idx, scaffold_length, data_top_dir, tokenizer, batch_size, device)

print("motif start indices", new_start_idx)
print("motif end indices", new_end_idx)


ALREADY DOWNLOADED
CLEANING PDB
sequence extracted from pdb YSSNTQQGRTSIVHLFEWRWVDIALECERYLAPKGFGGVQVSPPNENVAIHNPFRPWWERYQPVSYKLCTRSGNEDEFRNMVTRCNNVGVRIYVDAVINHMCGNAVSAGTSSTCGSYFNPGSRDFPAVPYSGWDFNDGKCKTGSGDIENYNDATQVRDCRLSGLLDLALGKDYVRSKIAEYMNHLIDIGVAGFRIDASKHMWPGDIKAILDKLHNLNSNWFPEGSKPFIYQEVIDLGGEPIKSSDYFGNGRVTEFKYGAKLGTVIRKWNGEKMSYLKNWGEGWGFMPSDRALVFVDNHDNQRGHGAGGASILTFWDARLYKMAVGFMLAHPYGFTRVMSSYRWPRYFENGKDVNDWVGPPNDNGVTKEVTINPDTTCGNDWVCEHRWRQIRNMVNFRNVVDGQPFTNWYDNGSNQVAFGRGNRGFIVFNNDDWTFSLTLQTGLPAGTYCDVISGDKINGNCTGIKIYVSDDGKAHFSISNSAEDPFIAIHAESKL
sequence length 495
motif extracted from indexes supplied: ASKHMWPGDIKAILDKLHNLNS
Generated sequence: ['MNFSSKDRGEFLTFAAKCGKKHKRKAIFVTGSRGTRSDEMAGYEQFRGKPNAIIGAGIIFLDEASKHMWPGDIKAILDKLHNLNSTTLIKEKCIEVCLEDIPAYGRDSWKLLLEDGGCPRNSGTKTEAIHHILELIELEQHEERTQLISYGRNLGLDDIIYGSDSEEEKAEE', 'MDDQTIQKKIAASLLSLILTTYEANEADTLTEENKNPIIDDEWARLFYLGLAILGKLSVKHDSASKHMWPGDIKAILDKLHNLNSNILANFFLQHICTLGKGQQKIKALDSALKAFTATLDSEEPTYRLGKKSNSGSTKVKKGAAPKSKPVTQRVEEDNTRGIV

In [29]:
#Save fasta files for these generated sequences:

file_paths=[] #A list to keep track of the paths to the fasta files

output_dir = 'data/interim/evo_generated_sequences/'
os.makedirs(output_dir, exist_ok=True)
for i, sequence in enumerate(generated_sequence):
    # Create a file path for each sequence file
    file_path = os.path.join(output_dir, f'sequence_{i}.fasta')
    file_paths.append(file_path) #Add this path to the list
    # Open the file in write mode and write the sequence in FASTA format
    with open(file_path, 'w') as f:
        f.write(f">SEQUENCE_{i}\n{sequence}\n")

    print(f"Generated sequence saved to: {file_path}")



Generated sequence saved to: data/interim/evo_generated_sequences/sequence_0.fasta
Generated sequence saved to: data/interim/evo_generated_sequences/sequence_1.fasta


Now we go back to the file saved by evodiff and we retrieve again the sequence that then will serve as input for omegafold

In [30]:
# Function to read the sequence from a FASTA file
def read_fasta(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # Assuming the sequence is in the second line of the FASTA file
        sequence = lines[1].strip()
    return sequence


# OmegaFold

## Installation

In [31]:

#import os,sys,re
import sys,re
from IPython.utils import io
if "SETUP_DONE" not in dir():
  import torch
  #Try to run on GPU, if it's available
  device = "cuda" if torch.cuda.is_available() else "cpu"
  with io.capture_output() as captured:
    #If OmegaFold is not already installed locally, install it
    if not os.path.isdir("OmegaFold"):
      %shell git clone --branch beta --quiet https://github.com/sokrypton/OmegaFold.git
      # %shell cd OmegaFold; pip -q install -r requirements.txt
      %shell pip -q install py3Dmol biopython==1.81
      %shell apt-get install aria2 -qq > /dev/null
      %shell aria2c -q -x 16 https://helixon.s3.amazonaws.com/release1.pt
      %shell mkdir -p ~/.cache/omegafold_ckpt
      %shell mv release1.pt ~/.cache/omegafold_ckpt/model.pt
  SETUP_DONE = True

In [32]:
import numpy as np

#@markdown ##Run **OmegaFold**
from string import ascii_uppercase, ascii_lowercase
import hashlib
import shutil

def get_hash(x): return hashlib.sha1(x.encode()).hexdigest()
alphabet_list = list(ascii_uppercase+ascii_lowercase)

def get_subbatch_size(L):
  if L <  500: return 500
  if L < 1000: return 200
  return 150

def renum_pdb_str(pdb_str, Ls=None, renum=True, offset=1):
  if Ls is not None:
    L_init = 0
    new_chain = {}
    for L,c in zip(Ls, alphabet_list):
      new_chain.update({i:c for i in range(L_init,L_init+L)})
      L_init += L

  n,num,pdb_out = 0,offset,[]
  resnum_ = None
  chain_ = None
  new_chain_ = new_chain[0]
  for line in pdb_str.split("\n"):
    if line[:4] == "ATOM":
      chain = line[21:22]
      resnum = int(line[22:22+5])
      if resnum_ is None: resnum_ = resnum
      if chain_ is None: chain_ = chain
      if resnum != resnum_ or chain != chain_:
        num += (resnum - resnum_)
        n += 1
        resnum_,chain_ = resnum,chain
      if Ls is not None:
        if new_chain[n] != new_chain_:
          num = offset
          new_chain_ = new_chain[n]
      N = num if renum else resnum
      if Ls is None: pdb_out.append("%s%4i%s" % (line[:22],N,line[26:]))
      else: pdb_out.append("%s%s%4i%s" % (line[:21],new_chain[n],N,line[26:]))
  return "\n".join(pdb_out)

pdb_strs = [] #A list for keeping track of the pdb strings, for visualization
fasta_fns = [] #A list for keeping track of the output fasta filenames
pdb_fns = [] #A list for keeping track of the output pdb filenames

#Loop through each of the created fasta files:
for i in np.arange(len(generated_sequence)):
  jobname = "evodiff_seq_" + str(i) #@param {type:"string"}
  jobname = re.sub(r'\W+', '', jobname)[:50]

  copies = 1 #@param {type:"integer"}
  #Note that "sequence" had been pulled directly from the fasta before, now make copies if needed
  sequence = read_fasta(file_paths[i])
  sequence = ":".join([sequence] * copies)



  output_path = "data/processed/"
  os.makedirs(output_path, exist_ok=True)

  #@markdown **Advanced Options**
  num_cycle = 1 #@param ["1", "2", "4", "8", "16", "32"] {type:"raw"}
  offset_rope = False #@param {type:"boolean"}

  ID = jobname+"_"+get_hash(sequence)[:5]
  seqs = sequence.split(":")
  lengths = [len(s) for s in seqs]

  subbatch_size = get_subbatch_size(sum(lengths))

  u_seqs = list(set(seqs))

  if len(seqs) == 1: mode = "mono"
  elif len(u_seqs) == 1: mode = "homo"
  else: mode = "hetero"

  with open(f"{ID}.fasta","w") as out:
    out.write(f">{ID}\n{sequence}\n")

  %shell python OmegaFold/main.py --offset_rope={offset_rope} --device={device} --subbatch_size={subbatch_size} --num_cycle={num_cycle} {ID}.fasta .

  pdb_str = renum_pdb_str(open(f"{ID}.pdb",'r').read(), Ls=lengths)
  with open(f"{ID}.pdb","w") as out:
    out.write(pdb_str)
  pdb_strs.append(pdb_str)

  #Now move these files to the desired output directory
  fasta_fn = str(ID) + ".fasta"
  pdb_fn = str(ID) + ".pdb"
  shutil.move(f"{ID}.fasta",output_path + fasta_fn)
  shutil.move(f"{ID}.pdb",output_path + pdb_fn)
  fasta_fns.append(fasta_fn) #Add the fasta filename to the list
  pdb_fns.append(pdb_fn) #Add the pdb filename to the list



/usr/local/lib/python3.10/dist-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(
/content/S2DS-Summer24-Polyploy/OmegaFold/omegafold/utils/protein_utils/residue_constants.py:483: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  eznorm = torch.cross(ex_normalized, ey_normalized)
INFO:root:Loading weights from /root/.cache/omegafold_ckpt/model.pt
INFO:root:Constructing OmegaFold
INFO:root:Reading evodiff_seq_0_2d04c.fasta
INFO:root:Predicting 1th chain in evodiff_seq_0_2d04c.fasta
INFO:root:172 residues in this chain.
INFO:root:Finished prediction in 179.47 seconds.
INFO:root:Saving predictio

## Visualise folded protein using Py3Dmol

If you have generated multiple proteins select which one you want to visualise in seq_idx by typing the index of the specific protein stored in pdb_strs.


In [33]:
#@markdown ##Display
import py3Dmol

seq_idx = 0 #@param {type:"integer"}

pymol_color_list = ["#33ff33","#00ffff","#ff33cc","#ffff00","#ff9999","#e5e5e5","#7f7fff","#ff7f00",
                    "#7fff7f","#199999","#ff007f","#ffdd5e","#8c3f99","#b2b2b2","#007fff","#c4b200",
                    "#8cb266","#00bfbf","#b27f7f","#fcd1a5","#ff7f7f","#ffbfdd","#7fffff","#ffff7f",
                    "#00ff7f","#337fcc","#d8337f","#bfff3f","#ff7fff","#d8d8ff","#3fffbf","#b78c4c",
                    "#339933","#66b2b2","#ba8c84","#84bf00","#b24c66","#7f7f7f","#3f3fa5","#a5512b"]

def show_pdb(pdb_str, show_sidechains=False, show_mainchains=False,
             color="pLDDT", chains=None, vmin=50, vmax=90,
             size=(800,480), hbondCutoff=4.0,
             Ls=None,
             animate=False):

  if chains is None:
    chains = 1 if Ls is None else len(Ls)
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=size[0], height=size[1])
  if animate:
    view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  else:
    view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  if color == "pLDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    HP = ["ALA","GLY","VAL","ILE","LEU","PHE","MET","PRO","TRP","CYS","TYR"]
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                  {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  view.zoomTo()
  if animate: view.animate()
  return view

color = "confidence" #@param ["confidence", "rainbow", "chain"]
if color == "confidence": color = "pLDDT"
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
show_pdb(pdb_strs[seq_idx], color=color, show_sidechains=show_sidechains, show_mainchains=show_mainchains,
         Ls=lengths).show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

#Run TMalign


To check if structure of scaffolded motif is kept in our generated protein.

This will output a TM-score between 0-1, where 1 is the best. Decide on a cut-off value (e.g. 0.9) to decide on "goodness" of protein.

In [34]:
# Step 1: Install necessary tools
!apt-get update
!apt-get install -y build-essential

# Step 2: Download the TM-align source code
!wget https://zhanglab.ccmb.med.umich.edu/TM-align/TMalign.cpp -O TMalign.cpp

# Step 3: Compile the source code
!g++ -O3 -ffast-math -lm -o TMalign TMalign.cpp


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [861 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelea

In [35]:
import re
import os
import subprocess
import glob

# Define the reference protein file path: it's where the motif pbd has been extracted to
ref_protein_abs = os.path.abspath(output_pdb_file)

# Use glob to find all PDB files in the omaga_fold output directory
generated_proteins = glob.glob(os.path.join(output_path, '*.pdb'))

# Loop through each generated protein file
for generated_protein in generated_proteins:
    generated_protein_abs = os.path.abspath(generated_protein)

    # Run the TMalign command
    tmalign_command = ["./TMalign", ref_protein_abs, generated_protein_abs]
    result = subprocess.run(tmalign_command, capture_output=True, text=True)

    # Print and save the results
    if result.returncode == 0:
        output = result.stdout
        print(f"Results for {generated_protein}:")
        print(output)

        # Extract TM-scores from the output using regex
        tm_scores = re.findall(r"TM-score=\s+([\d.]+)", output)
        if tm_scores:
            tm_score_1 = float(tm_scores[0])
            tm_score_2 = float(tm_scores[1])
            print(f"TM-score: {tm_score_1}, if lower than 0.9 run pipeline again")

            # Use tm_score_1 as needed in subsequent commands
        else:
            print("TM-score not found in the output.")
    else:
        print(f"Error running TMalign for {generated_protein}. Return code: {result.returncode}")
        print(result.stderr)


Results for data/processed/evodiff_seq_0_be355.pdb:

 *********************************************************************
 * TM-align (Version 20220412): protein structure alignment          *
 * References: Y Zhang, J Skolnick. Nucl Acids Res 33, 2302-9 (2005) *
 * Please email comments and suggestions to zhanglab@zhanggroup.org   *
 *********************************************************************

Name of Chain_1: /content/S2DS-Summer24-Polyploy/data/interim/test_extracted_motif.pdb (to be superimposed onto Chain_2)
Name of Chain_2: /content/S2DS-Summer24-Polyploy/data/processed/evodiff_seq_0_be355.pdb
Length of Chain_1: 22 residues
Length of Chain_2: 172 residues

Aligned length= 20, RMSD=   2.84, Seq_ID=n_identical/n_aligned= 0.150
TM-score= 0.19861 (if normalized by length of Chain_1, i.e., LN=22, d0=0.57)
TM-score= 0.09244 (if normalized by length of Chain_2, i.e., LN=172, d0=4.89)
(You should use TM-score normalized by length of the reference structure)

(":" denotes resi